# Ensemble Learning

## Initial Imports

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter

In [3]:
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

## Read the CSV and Perform Basic Data Cleaning

In [4]:
# Load the data
file_path = Path('Resources/LoanStats_2019Q1.csv')
df = pd.read_csv(file_path)

df = pd.get_dummies(df, columns=["home_ownership", "verification_status","pymnt_plan", "hardship_flag", "debt_settlement_flag", "initial_list_status", "application_type"])

datetime = pd.to_datetime(df['issue_d'])
issue_year = pd.DatetimeIndex(df['issue_d']).year
issue_month = pd.DatetimeIndex(df['issue_d']).month
payment_year = pd.DatetimeIndex(df['next_pymnt_d']).year
payment_month = pd.DatetimeIndex(df['next_pymnt_d']).month

df = df.drop(columns="issue_d")
df = df.drop(columns="next_pymnt_d")
df['issue_year']=issue_year
df['issue_month']=issue_month
df['payment_year']=payment_year
df['payment_month']=payment_month

# Preview the data
df.head()

,loan_amnt,int_rate,installment,annual_inc,loan_status,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,...,hardship_flag_N,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,issue_year,issue_month,payment_year,payment_month
0,10500.0,0.1719,375.35,66000.0,low_risk,27.24,0.0,0.0,8.0,0.0,...,1,1,0,1,1,0,2019,3,2019,5
1,25000.0,0.2000,929.09,105000.0,low_risk,20.23,0.0,0.0,17.0,1.0,...,1,1,0,1,1,0,2019,3,2019,5
2,20000.0,0.2000,529.88,56000.0,low_risk,24.26,0.0,0.0,8.0,0.0,...,1,1,0,1,1,0,2019,3,2019,5
3,10000.0,0.1640,353.55,92000.0,low_risk,31.44,0.0,1.0,10.0,1.0,...,1,1,0,1,1,0,2019,3,2019,5
4,22000.0,0.1474,520.39,52000.0,low_risk,18.76,0.0,1.0,14.0,0.0,...,1,1,0,1,1,0,2019,3,2019,5


In [5]:
file_path = Path("Resources/lol.csv")
df.to_csv(file_path, index=False)

## Split the Data into Training and Testing

In [6]:
# Create our features
X = df.drop(columns="loan_status")

# Create our target
y = df["loan_status"]

In [7]:
X.describe()

,loan_amnt,int_rate,installment,annual_inc,dti,delinq_2yrs,inq_last_6mths,open_acc,pub_rec,revol_bal,...,hardship_flag_N,debt_settlement_flag_N,initial_list_status_f,initial_list_status_w,application_type_Individual,application_type_Joint App,issue_year,issue_month,payment_year,payment_month
count,68817.000000,68817.000000,68817.000000,6.881700e+04,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,68817.000000,...,68817.0,68817.0,68817.000000,68817.000000,68817.000000,68817.000000,68817.0,68817.000000,68817.0,68817.000000
mean,16677.594562,0.127718,480.652863,8.821371e+04,21.778153,0.217766,0.497697,12.587340,0.126030,17604.142828,...,1.0,1.0,0.123879,0.876121,0.860340,0.139660,2019.0,1.726172,2019.0,4.616839
std,10277.348590,0.048130,288.062432,1.155800e+05,20.199244,0.718367,0.758122,6.022869,0.336797,21835.880400,...,0.0,0.0,0.329446,0.329446,0.346637,0.346637,0.0,0.743862,0.0,0.486161
min,1000.000000,0.060000,30.890000,4.000000e+01,0.000000,0.000000,0.000000,2.000000,0.000000,0.000000,...,1.0,1.0,0.000000,0.000000,0.000000,0.000000,2019.0,1.000000,2019.0,4.000000
25%,9000.000000,0.088100,265.730000,5.000000e+04,13.890000,0.000000,0.000000,8.000000,0.000000,6293.000000,...,1.0,1.0,0.000000,1.000000,1.000000,0.000000,2019.0,1.000000,2019.0,4.000000
50%,15000.000000,0.118000,404.560000,7.300000e+04,19.760000,0.000000,0.000000,11.000000,0.000000,12068.000000,...,1.0,1.0,0.000000,1.000000,1.000000,0.000000,2019.0,2.000000,2019.0,5.000000
75%,24000.000000,0.155700,648.100000,1.040000e+05,26.660000,0.000000,1.000000,16.000000,0.000000,21735.000000,...,1.0,1.0,0.000000,1.000000,1.000000,0.000000,2019.0,2.000000,2019.0,5.000000
max,40000.000000,0.308400,1676.230000,8.797500e+06,999.000000,18.000000,5.000000,72.000000,4.000000,587191.000000,...,1.0,1.0,1.000000,1.000000,1.000000,1.000000,2019.0,3.000000,2019.0,5.000000


In [8]:
# Check the balance of our target values
# YOUR CODE HERE
y.value_counts()

low_risk     68470
high_risk      347
Name: loan_status, dtype: int64

In [9]:
# Split the X and y into X_train, X_test, y_train, y_test
# YOUR CODE HERE
X_train, X_test, y_train, y_test = train_test_split(X, 
                                                    y, 
                                                    random_state=1, 
                                                    stratify=y)

## Data Pre-Processing

Scale the training and testing data using the `StandardScaler` from `sklearn`. Remember that when scaling the data, you only scale the features data (`X_train` and `X_testing`).

In [10]:
# Create the StandardScaler instance
from sklearn.preprocessing import StandardScaler
# YOUR CODE HERE
scaler = StandardScaler()

In [11]:
# Fit the Standard Scaler with the training data
# When fitting scaling functions, only train on the training dataset
# YOUR CODE HERE
X_scaler = scaler.fit(X_train)

In [12]:
# Scale the training and testing data
# YOUR CODE HERE
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

## Ensemble Learners

In this section, you will compare two ensemble algorithms to determine which algorithm results in the best performance. You will train a Balanced Random Forest Classifier and an Easy Ensemble classifier . For each algorithm, be sure to complete the folliowing steps:

1. Train the model using the training data. 
2. Calculate the balanced accuracy score from sklearn.metrics.
3. Display the confusion matrix from sklearn.metrics.
4. Generate a classication report using the `imbalanced_classification_report` from imbalanced-learn.
5. For the Balanced Random Forest Classifier only, print the feature importance sorted in descending order (most important feature to least important) along with the feature score

Note: Use a random state of 1 for each algorithm to ensure consistency between tests

### Balanced Random Forest Classifier

In [13]:
from imblearn.ensemble import BalancedRandomForestClassifier
# Resample the training data with the BalancedRandomForestClassifier
# YOUR CODE HERE
brf = BalancedRandomForestClassifier(n_estimators=1000, random_state=1)
brf.fit(X_train, y_train)

BalancedRandomForestClassifier(n_estimators=1000, random_state=1)

In [14]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = brf.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7967857991789243

In [15]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[   59,    28],
       [ 1448, 15670]], dtype=int64)

In [16]:
# Print the imbalanced classification report
# YOUR CODE HERE
y_pred_rf = brf.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred_rf))

                   pre       rec       spe        f1       geo       iba       sup

  high_risk       0.04      0.68      0.92      0.07      0.79      0.61        87
   low_risk       1.00      0.92      0.68      0.96      0.79      0.64     17118

avg / total       0.99      0.91      0.68      0.95      0.79      0.64     17205



In [17]:
# List the features sorted in descending order by feature importance
# YOUR CODE HERE
importances = brf.feature_importances_
sorted(zip(brf.feature_importances_, X.columns), reverse=True)

[(0.07088423403187716, 'total_rec_prncp'),
 (0.06345161379945045, 'total_pymnt'),
 (0.0613550620563151, 'last_pymnt_amnt'),
 (0.06055928741032528, 'total_rec_int'),
 (0.05391850406543055, 'total_pymnt_inv'),
 (0.02816171681314045, 'issue_month'),
 (0.026605431341876585, 'int_rate'),
 (0.01869055279017164, 'dti'),
 (0.017873845862634196, 'installment'),
 (0.017608912755844985, 'annual_inc'),
 (0.016023878012560958, 'mths_since_recent_inq'),
 (0.015523218594663202, 'revol_bal'),
 (0.015459219116168472, 'out_prncp'),
 (0.015378978734975365, 'out_prncp_inv'),
 (0.015125968368010357, 'max_bal_bc'),
 (0.01495473559859488, 'all_util'),
 (0.014941086406176127, 'tot_hi_cred_lim'),
 (0.014931095519459554, 'avg_cur_bal'),
 (0.014670241706162154, 'mo_sin_old_rev_tl_op'),
 (0.01461104539915508, 'bc_util'),
 (0.01450713693069814, 'il_util'),
 (0.01448625242192749, 'total_bc_limit'),
 (0.01441701706096177, 'mo_sin_old_il_acct'),
 (0.013747092307394643, 'bc_open_to_buy'),
 (0.013552092080031306, 'tota

### Easy Ensemble Classifier

In [18]:
from imblearn.ensemble import EasyEnsembleClassifier
# Train the Classifier
# YOUR CODE HERE
eec = EasyEnsembleClassifier(n_estimators=1000, random_state=1)
eec.fit(X_train, y_train)

EasyEnsembleClassifier(n_estimators=1000, random_state=1)

In [19]:
# Calculated the balanced accuracy score
# YOUR CODE HERE
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9266541370044035

In [20]:
# Display the confusion matrix
# YOUR CODE HERE
confusion_matrix(y_test, y_pred)

array([[   79,     8],
       [  937, 16181]], dtype=int64)

In [ ]:
# Print the imbalanced classification report
# YOUR CODE HERE
y_pred_rf = eec.predict(X_test)
print(classification_report_imbalanced(y_test, y_pred_rf))

### Final Questions

1. Which model had the best balanced accuracy score?

   The Easy Ensemble Classifier had the best balanced accuracy score at 0.93.

2. Which model had the best recall score?

   The Easy Ensemble Classifier had the best balanced accuracy score at 0.93.

3. Which model had the best geometric mean score?

   The Easy Ensemble Classifier had the best geometric mean score.

4. What are the top three features?

   1: Total rec principle,
   2: Total payment,
   3: Last payment amount,